In [22]:
import os
import json
import base64
from tqdm import tqdm


In [23]:
# 폴더를 가져옵니다.
raw_data_folder = r"C:\Users\Smart\Desktop\datas\emergency_vehicle\emergency_labeling_data\ambulance\labeling_data"
output_folder = r"C:\Users\Smart\Desktop\code\ai_coverted"
img_folder = r"C:\Users\Smart\Desktop\datas\emergency_vehicle\emergency_img"

# 폴더의 파일 목록을 가져옵니다.
files = os.listdir(raw_data_folder)

# 폴더의 파일 목록을 순회합니다.
for file in tqdm(files):
    # 파일 이름을 가져옵니다.
    filename = file.split(".")[0]

    # desktop.ini 파일이면 건너뜁니다.
    if file.lower() == "desktop.ini":
        continue

    # 이미지 파일을 열어서 base64로 인코딩합니다.
    image_path = os.path.join(img_folder, filename + ".jpg")
    with open(image_path, "rb") as img_file:
        image_data = base64.b64encode(img_file.read()).decode()

    # 파일을 읽습니다.
    with open(os.path.join(raw_data_folder, file), "r", encoding="UTF-8") as f:
        data = json.load(f)

    labels = []

    for obj in data["learningDataInfo"]["objects"]:
        annotation = obj["annotation"]
        label = data["rawDataInfo"]["SmallCategoryId"]

        if label == "구급차":
            label = "ambulance"

        if annotation == "bbox":
            tl_x = obj["coords"]["tl"]["x"]
            tl_y = obj["coords"]["tl"]["y"]
            br_x = obj["coords"]["br"]["x"]
            br_y = obj["coords"]["br"]["y"]
            
            labels.append({
                "label": label,
                "points":[ 
                        [tl_x, tl_y],
                        [br_x, br_y]
                ],
                "group_id": None,
                "description": "",
                "shape_type": "rectangle",
                "flags": {}
            })

    annotation = {
        "version": "5.2.1",
        "flags": {},
        "shapes": labels,
        "imagePath": filename + ".jpg",
        "imageData": image_data,
        "imageHeight": 1080,
        "imageWidth": 1920
    }

    # 라벨링 데이터를 다른 폴더에 .json 형식으로 저장합니다.
    with open(os.path.join(output_folder, filename + ".json"), "w", encoding='UTF-8') as f:
        json.dump(annotation, f, ensure_ascii=False, indent=4)

  0%|          | 0/4578 [00:00<?, ?it/s]

100%|██████████| 4578/4578 [04:10<00:00, 18.25it/s]
